In [8]:
%matplotlib widget 

import ipywidgets as widgets
from IPython.display import display
import Berthing
import Catalogue
from SCN import SCN
from MV import MV
from Pnuematic import Pnuematic
import numpy as np

def CreateComboBox(description, placeholder, options):
    wget = widgets.Combobox(
    placeholder=placeholder,
    options=options,
    description=description,
    ensure_option=True,
    disabled=False
    )   
    return wget

def CreateToggleButtons(description, options):
    wget = widgets.ToggleButtons(
        description=description,
        options=options,
        disabled = False,
        button_style = ''
    )
    return wget

def CreateSelect(description, options, default_val=None, style=None):
    
    if style is None:
        style = {
            'description_width':'initial'
        }        
   
    wget = widgets.Select(
        description=description,
        options=options,
        value=default_val,
        disabled=False,
        style=style
    )
    return wget

def CreateFloatText(description, default_val, style=None):

    if style is None:
        style = {
            'description_width':'initial'
        }    

    wget = widgets.FloatText(
        description=description,
        value=default_val,
        style=style,
    )

    return wget

def CreateIntText(description, default_val, style=None):

    if style is None:
        style = {
            'description_width':'initial'
        }    

    wget = widgets.IntText(
        description=description,
        value=default_val,
        style=style
    )

    return wget

def CreateButton(description, style=None):
    
    if style is None:
        style = {
            'description_width':'initial'
        }        

    wget = widgets.Button(
        description=description,
        disabled = False,
        style=style
    )
    return wget

def CreateSelectionSlider(description, options, default_val, style=None):

    if style is None:
        style = {
            'description_width':'initial'
        }

    wget = widgets.SelectionSlider(
        options=options,
        value=default_val,
        description=description,
        style=style,
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True
    )
    return wget

def CreateDropdown(description, options, default_val, style=None):
     
    if style is None:
        style = {
            'description_width':'initial'
        }        
   
    wget = widgets.Dropdown(
        description=description,
        options=options,
        value=default_val,
        disabled=False,
        style=style
    )
    return wget

def CreateRadioButtons(description, options, default_val):

    wget = widgets.RadioButtons(
        description=description,
        options=options,
        value=default_val,
        disabled=False
    )

    return wget

def DisplayWidget(widgets:list):
    
    if widgets is list:
        for w in widgets:
            display(w)
        return
    else:
        display(widgets)



# Berthing Energy calculation

In [9]:

mass_calc_w = CreateDropdown('Mass coefficient method',['Vasco Costa','Shigeru','PIANC'],'Vasco Costa')
DisplayWidget(mass_calc_w)

berthing_pt_options = [
    "1/6 point",
    "1/5 point",
    "1/4 point",
    "1/3 point"
]

style = {
    "description_width": "intial"
}

#Berthing event parameters
berthing_angle_w = CreateFloatText("Berthing angle [°]:", 0.0)
velocity_w = CreateFloatText("Berthing velocity [m/s]:", 0.01)
velocity_angle_w = CreateFloatText("Velocity angle [°]:", 0.0)
berthing_point_w = CreateSelectionSlider("Berthing point:",berthing_pt_options, "1/4 point")

berthing_event_container = widgets.VBox([berthing_angle_w, velocity_w, velocity_angle_w, berthing_point_w])

#Vessel parameters
displacement_w = CreateIntText("Displacement [t]:", 0)
LBP_w = CreateFloatText("LBP [m]:",0.0)
beam_w = CreateFloatText("Beam [m]:",0.0)
draft_w = CreateFloatText("Draft [m]:",0.0)
UKC_w = CreateFloatText("UKC [m]:",0.0)
hull_radius_w = CreateFloatText("Hull radius [m]:", 0.0)

vessel_container = widgets.VBox([displacement_w,LBP_w, beam_w, draft_w, UKC_w, hull_radius_w])

#Berthing coefficients
softness_coefficient_w = CreateFloatText("Softness coefficient [Cs]:", 1.0)
configuration_coefficient_w = CreateFloatText("Configuration coefficient [Cc]:",1.0)
abnormal_factor_w = CreateFloatText("Abnormal Factor:", 2.0)
coefficients_container = widgets.VBox([abnormal_factor_w,softness_coefficient_w,configuration_coefficient_w])

#Accordian
berthing_container = widgets.Accordion(children=[berthing_event_container, vessel_container, coefficients_container])

berthing_container.set_title(0, "Berthing event")
berthing_container.set_title(1, "Vessel particulars")
berthing_container.set_title(2, "Berthing coefficients")
DisplayWidget(berthing_container)


Dropdown(description='Mass coefficient method', options=('Vasco Costa', 'Shigeru', 'PIANC'), style=Description…

Accordion(children=(VBox(children=(FloatText(value=0.0, description='Berthing angle [°]:', style=DescriptionSt…

In [10]:
berthing_point = {
    "1/6 point": 0.333,
    "1/5 point": 0.3,
    "1/4 point": 0.25,
    "1/3 point": 0.1667
}

berthing_point_val = berthing_point.get(berthing_point_w.value)*LBP_w.value

energy_output = widgets.Output()
design_energy = 0

@energy_output.capture(clear_output=True, wait=True)
def BerthingEnergy(b):
    try:
        abnormal_factor = abnormal_factor_w.value
        normal_energy = Berthing.BerthingEnergy(
            berthing_velocity=velocity_w.value,
            berthing_angle=berthing_angle_w.value,
            velocity_angle=velocity_angle_w.value,
            displacement=displacement_w.value,
            beam=beam_w.value,
            LBP=LBP_w.value,
            draft=draft_w.value,
            UKC=UKC_w.value,
            berthing_point=berthing_point_val,
            #hull_radius=hull_radius_w.value,
            softness_coefficient=softness_coefficient_w.value,
            configuration_coefficient=configuration_coefficient_w.value,
            mass_calc=mass_calc_w.value,
            output=False
        )
        design_energy = normal_energy*abnormal_factor

        print(f"Normal energy: {normal_energy:.2f}kNm")
        print(f"Abnormal energy: {design_energy:.2f}kNm")
    except ZeroDivisionError:
        print("Design inputs are incomplete. Review chosen values.")


calc_energy_btn = CreateButton("Calculate")
calc_energy_btn.on_click(BerthingEnergy)
DisplayWidget(calc_energy_btn)
display(energy_output)
#DisplayWidget([calc_btn,energy_output])

Button(description='Calculate', style=ButtonStyle())

Output()

# Fender design

In [11]:
fender_types = ["SCN","MV","Pnuematic"]

# Universal parameters
berthing_energy_w = CreateFloatText("Berthing Energy:",0)
manufacturing_tolerance_w = CreateFloatText("Manufacturing Tolerance [%]:",10)
max_temp_w = CreateFloatText("Max Temperature [°C]:",23.0)
min_temp_w = CreateFloatText("Min Temperature [°C]:",23.0)
calc_btn = CreateButton("Calculate")
design_parameters = widgets.HBox([berthing_energy_w, manufacturing_tolerance_w, max_temp_w, min_temp_w])
DisplayWidget(design_parameters)
DisplayWidget(calc_btn)

# SCN
SCN_w = CreateSelect('Chose Fender:',Catalogue.SCN,'SCN300')
SCN_grade_w = CreateSelect('Chose Fender Grade:',Catalogue.SCN_grades,0.9)
SCN_material_w = CreateSelect('Chose fender material:', ["Blend", "Rubber", "Synthetic"],"Blend")

SCN_container = widgets.HBox([SCN_w,SCN_grade_w, SCN_material_w])

# MV
MV_w = CreateSelect('Chose Fender:',Catalogue.MV,"300x600")
MV_compound_w = CreateSelect('Chose Compound Type:',['A','B'], 'A')
MV_unit_spacing_w = CreateIntText("Unit spacing [mm]:", 0)
MV_bow_flare_w = CreateFloatText("Bow Flare angle [°]:", 0.0)

MV_container = widgets.HBox([MV_w,MV_compound_w,MV_unit_spacing_w,MV_bow_flare_w])

# Pnuematic
pnuematic_w = CreateSelect('Chose Fender:', Catalogue.pnuematic, "500x1000")
pnuematic_pressure_w = CreateSelect('Chose inflation pressure:', [50, 80], 50)

Pnuematic_container = widgets.HBox([pnuematic_w,pnuematic_pressure_w])

fender_tab = widgets.Tab(children=[SCN_container, MV_container, Pnuematic_container])
for i in range(len(fender_tab.children)):
    fender_tab.set_title(i, fender_types[i])

DisplayWidget(fender_tab)

Button(description='Calculate', style=ButtonStyle())

In [23]:
output = widgets.Output()

@output.capture(clear_output=True, wait=True)
def FenderChart(b):
    fender = FenderCalc()
    berthing_energy = berthing_energy_w.value
    berthing_angle = berthing_angle_w.value
    velocity = velocity_w.value
    max_temp = max_temp_w.value
    min_temp = min_temp_w.value

    design_fender = fender_types[fender_tab.selected_index]

    #TODO add back in capacity factor. It's in SCN section below
    if berthing_energy > fender.rated_energy:
        print("Rated capacity of the fender is exceeded. Select another size or grade.")
    elif design_fender == "MV":
        flare_angle = MV_bow_flare_w.value
        fender.fender_chart(berthing_angle, flare_angle, velocity, max_temp, min_temp, berthing_energy)
    elif design_fender == "Pnuematic":
        fender.fender_chart(berthing_energy)
    else:
        fender.fender_chart(berthing_angle, velocity, max_temp, min_temp, berthing_energy) 
    
def FenderCalc():

    fender_type = fender_types[fender_tab.selected_index]
    manufacturing_tolerance = manufacturing_tolerance_w.value/100

    if fender_type == 'SCN':
        size = SCN_w.value
        grade = SCN_grade_w.value
        material = SCN_material_w.value
        fender = SCN(size,
                     grade,
                     manufacturing_tolerance,
                     manufacturing_tolerance, 
                     material)
        #energy_factor, reaction_factor = fender.capacity_factor(berthing_angle, velocity, temp)
        return fender
        
    elif fender_type == 'MV':
        size = MV_w.value
        compound = MV_compound_w.value
        leg_spacing = MV_unit_spacing_w.value

        fender = MV(size,
                    compound,
                    leg_spacing,
                    manufacturing_tolerance,
                    manufacturing_tolerance)
        return fender
        
    elif fender_type == 'Pnuematic':
        size = pnuematic_w.value
        pressure = pnuematic_pressure_w.value
        fender = Pnuematic(size,
                           0,
                           manufacturing_tolerance,
                           pressure)
        return fender
        
calc_btn.on_click(FenderChart)
display(output)

Output()

In [24]:
selection_w = CreateSelect('',[])

def CriteriaChange(change):
    design_criteria = design_criteria_w.value
    fender_type =fender_types[fender_tab.selected_index]
       
    if design_criteria == 'Depth':
        selection_w.description = "Select fender:"
    else:
        selection_w.description = "Select grade:"
    if fender_type == 'SCN':
        if design_criteria == 'Depth':
            selection_w.options = Catalogue.SCN
        else:
            selection_w.options = Catalogue.SCN_grades  
    elif fender_type == 'MV':
        if design_criteria == 'Depth':
            selection_w.options = Catalogue.MV
        else:    
            selection_w.options = ['A','B']
    elif fender_type == 'Pnuematic':
        if design_criteria == 'Depth':
            selection_w.options = Catalogue.pnuematic
        else:
            selection_w.options = [50, 80]
    

design_criteria_w = CreateRadioButtons('Design criteria', ['Depth','Grade'],'Depth')
design_criteria_w.observe(CriteriaChange, 'value')
fender_tab.observe(CriteriaChange,'value')

DisplayWidget(design_criteria_w)
DisplayWidget(selection_w)
CriteriaChange(True)

#selection_containter = widgets.HBox([])
#DisplayWidget(selection_containter)

RadioButtons(description='Design criteria', options=('Depth', 'Grade'), value='Depth')

Select(options=(), style=DescriptionStyle(description_width='initial'), value=None)

In [26]:
optimal_fender_w = widgets.Output()

def first_index(val, search_list):
    index = next(
        (index for index, item in enumerate(search_list) if item > val), None
    )
    return index

@optimal_fender_w.capture(clear_output=True, wait=True)
def OptimalFender(b):
    berthing_energy = berthing_energy_w.value
    design_criteria = design_criteria_w.value
    selection = selection_w.value
    fender_type = fender_types[fender_tab.selected_index]    

    berthing_angle = berthing_angle_w.value
    velocity = velocity_w.value
    max_temp = max_temp_w.value
    
    fender = FenderCalc()
    
    if fender_type == 'SCN':
        energy_capacities = np.array(Catalogue.SCN_ratings[1])

        angular_factor = fender.angle_factor(berthing_angle)[0]
        velocity_factor = fender.velocity_factor(velocity)
        temperature_factor = fender.temperature_factor(max_temp)

        reduction_factor = angular_factor*velocity_factor*temperature_factor

        if design_criteria == 'Depth':
            row = Catalogue.SCN.index(selection)
            design_energies = energy_capacities[row,:]*reduction_factor
            col = first_index(berthing_energy, design_energies)
            if col is not None: 
                print(f"{selection}-F{Catalogue.SCN_grades[col]}")
            else:
                print("This size is not sufficient, select larger fender.")
        else:
            col = Catalogue.SCN_grades.index(selection)
            design_energies = energy_capacities[:,col]*reduction_factor
            row = first_index(berthing_energy, design_energies)
            if row is not None:
                print(f"{Catalogue.SCN[row]}-F{selection}")
            else: 
                print(" This grade is not sufficient, select higher grade.")


    elif fender_type == 'MV':
        if design_criteria == 'Depth':
            energy_capacities_A = Catalogue.MV_compound_A
            energy_capacities_B = Catalogue.MV_compound_B

        else:    
            iter_list = Catalogue.MV
    elif fender_type == 'Pnuematic':
        if design_criteria == 'Depth':
            iter_list = [50, 80]    
        else:
            iter_list = Catalogue.pnuematic

fender_calc_btn = CreateButton("Calculate")
fender_calc_btn.on_click(OptimalFender)

DisplayWidget(fender_calc_btn)
display(optimal_fender_w)

Button(description='Calculate', style=ButtonStyle())

Output()